In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import os
from IPython.display import IFrame, display
import glob

yearly_folder = r'D:yourfilelocation'
all_files = sorted(glob.glob(os.path.join(yearly_folder, '*.csv')))
df_list = []
for file in all_files:
    df_temp = pd.read_csv(file)
    df_temp['time'] = pd.to_datetime(df_temp['time'], utc=True)
    df_temp['time'] = df_temp['time'].dt.tz_convert('America/New_York')
    df_list.append(df_temp)
df = pd.concat(df_list, ignore_index=True)
df = df.sort_values('time').reset_index(drop=True)

def build_sessions(df):
    all_dates = pd.date_range(df['time'].dt.date.min(), df['time'].dt.date.max(), freq='D', tz='America/New_York')
    sessions = []
    for d in all_dates:
        weekday = d.weekday()
        if weekday == 0:
            session_start = pd.Timestamp.combine(d - pd.Timedelta(days=1), pd.to_datetime('18:00:00').time()).tz_localize('America/New_York')
            session_end = pd.Timestamp.combine(d, pd.to_datetime('16:59:00').time()).tz_localize('America/New_York')
            sessions.append({'session_date': d.date(), 'start': session_start, 'end': session_end})
        elif weekday in [1, 2, 3, 4]:
            session_start = pd.Timestamp.combine(d - pd.Timedelta(days=1), pd.to_datetime('18:00:00').time()).tz_localize('America/New_York')
            session_end = pd.Timestamp.combine(d, pd.to_datetime('16:59:00').time()).tz_localize('America/New_York')
            sessions.append({'session_date': d.date(), 'start': session_start, 'end': session_end})
    return pd.DataFrame(sessions)

sessions_df = build_sessions(df)

df = df.sort_values('time')
sessions_df = sessions_df.sort_values('start')
df = pd.merge_asof(
    df,
    sessions_df[['session_date', 'start', 'end']],
    left_on='time',
    right_on='start',
    direction='backward',
    tolerance=pd.Timedelta('24h')
)
df = df[(df['time'] >= df['start']) & (df['time'] <= df['end'])]
df['futures_session'] = df['session_date']

pio.renderers.default = 'iframe'

save_dir = r'D:yourfilelocation'
os.makedirs(save_dir, exist_ok=True)

unique_sessions = sessions_df['session_date'].tolist()

pd_levels = []
for session_date in unique_sessions:
    rth_start = pd.Timestamp.combine(session_date, pd.to_datetime('09:30:00').time()).tz_localize('America/New_York')
    rth_end = pd.Timestamp.combine(session_date, pd.to_datetime('16:14:00').time()).tz_localize('America/New_York')
    rth_df = df[(df['time'] >= rth_start) & (df['time'] <= rth_end)]
    pd_levels.append({
        'session_date': session_date,
        'high': rth_df['high'].max() if not rth_df.empty else np.nan,
        'low': rth_df['low'].min() if not rth_df.empty else np.nan
    })
pd_levels_df = pd.DataFrame(pd_levels)

rth_gap_rows = []
for idx, session_date in enumerate(unique_sessions):
    if idx == 0:
        continue
    prev_session = unique_sessions[idx - 1]
    prev_close_time = pd.Timestamp.combine(prev_session, pd.to_datetime('16:14:00').time()).tz_localize('America/New_York')
    prev_close_bar = df[df['time'] == prev_close_time]
    if prev_close_bar.empty:
        continue
    prev_close = prev_close_bar['close'].iloc[0]
    open_0930_time = pd.Timestamp.combine(session_date, pd.to_datetime('09:30:00').time()).tz_localize('America/New_York')
    open_0930_bar = df[df['time'] == open_0930_time]
    if open_0930_bar.empty:
        continue
    open_0930 = open_0930_bar['open'].iloc[0]
    rth_gap = open_0930 - prev_close
    rth_gap_rows.append({
        'session_date': session_date,
        'open_0930_time': open_0930_time,
        'open_0930': open_0930,
        'prev_close_time': prev_close_time,
        'prev_close': prev_close,
        'rth_gap': rth_gap
    })
rth_gap_df = pd.DataFrame(rth_gap_rows)

for idx, session_date in enumerate(unique_sessions):
    session_row = sessions_df[sessions_df['session_date'] == session_date].iloc[0]
    session_start_dt = session_row['start']
    session_end_dt = session_row['end']
    session_df = df[(df['time'] >= session_start_dt) & (df['time'] <= session_end_dt)]

    shapes = [
        dict(
            type="rect",
            xref="paper",
            yref="paper",
            x0=0,
            y0=0,
            x1=1,
            y1=1,
            line=dict(
                color="Black",
                width=1
            )
        )
    ]

    annotations = []
    prev_close = None
    if idx > 0:
        prev_session = unique_sessions[idx - 1]
        prev_row = sessions_df[sessions_df['session_date'] == prev_session].iloc[0]
        prev_session_start_dt = prev_row['start']
        prev_session_end_dt = prev_row['end']
        prev_session_df = df[(df['time'] >= prev_session_start_dt) & (df['time'] <= prev_session_end_dt)]
        prev_close_time = pd.Timestamp.combine(prev_session, pd.to_datetime('16:14:00').time()).tz_localize('America/New_York')
        prev_close_bar = prev_session_df[prev_session_df['time'] == prev_close_time]
        if not prev_close_bar.empty:
            prev_close = prev_close_bar['close'].iloc[0]
            shapes.append(dict(
                type="line",
                xref="x",
                yref="y",
                x0=session_df['time'].iloc[0] if not session_df.empty else session_start_dt,
                y0=prev_close,
                x1=session_df['time'].iloc[-1] if not session_df.empty else session_end_dt,
                y1=prev_close,
                line=dict(color="#ff9800", width=1, dash="solid"),
                layer="above"
            ))
            session_9am = pd.Timestamp.combine(session_date, pd.to_datetime('12:00:00').time()).tz_localize('America/New_York')
            annotations.append(dict(
                x=session_9am,
                y=prev_close,
                xref="x",
                yref="y",
                text="Prev Day Close",
                showarrow=False,
                font=dict(color="#000000", size=10),
                xanchor="left",
                yanchor="bottom"
            ))

    open_0930 = None
    session_9am = pd.Timestamp.combine(session_date, pd.to_datetime('12:00:00').time()).tz_localize('America/New_York')
    open_0930_time = pd.Timestamp.combine(session_date, pd.to_datetime('09:30:00').time()).tz_localize('America/New_York')
    open_0930_bar = session_df[session_df['time'] == open_0930_time]
    if not open_0930_bar.empty:
        open_0930 = open_0930_bar['open'].iloc[0]
        shapes.append(dict(
            type="line",
            xref="x",
            yref="y",
            x0=session_df['time'].iloc[0] if not session_df.empty else session_start_dt,
            y0=open_0930,
            x1=session_df['time'].iloc[-1] if not session_df.empty else session_end_dt,
            y1=open_0930,
            line=dict(color="#ff9800", width=1, dash="solid"),
            layer="above"
        ))
        annotations.append(dict(
            x=session_9am,
            y=open_0930,
            xref="x",
            yref="y",
            text="09:30 Open",
            showarrow=False,
            font=dict(color="#000000", size=10),
            xanchor="left",
            yanchor="bottom"
        ))

    rth_gap_range = None
    if prev_close is not None and open_0930 is not None:
        rth_gap_range = open_0930 - prev_close
        line_025 = prev_close + 0.25 * rth_gap_range
        line_050 = prev_close + 0.50 * rth_gap_range
        line_075 = prev_close + 0.75 * rth_gap_range

        shapes.append(dict(
            type="line",
            xref="x",
            yref="y",
            x0=session_df['time'].iloc[0] if not session_df.empty else session_start_dt,
            y0=line_025,
            x1=session_df['time'].iloc[-1] if not session_df.empty else session_end_dt,
            y1=line_025,
            line=dict(color="#ff9800", width=1, dash="dash"),
            layer="above"
        ))
        annotations.append(dict(
            x=session_9am,
            y=line_025,
            xref="x",
            yref="y",
            text="0.25% RTH Gap",
            showarrow=False,
            font=dict(color="#000000", size=10),
            xanchor="left",
            yanchor="bottom"
        ))

        shapes.append(dict(
            type="line",
            xref="x",
            yref="y",
            x0=session_df['time'].iloc[0] if not session_df.empty else session_start_dt,
            y0=line_050,
            x1=session_df['time'].iloc[-1] if not session_df.empty else session_end_dt,
            y1=line_050,
            line=dict(color="#ffffff", width=1, dash="dash"),
            layer="above"
        ))
        annotations.append(dict(
            x=session_9am,
            y=line_050,
            xref="x",
            yref="y",
            text="0.5% RTH Gap",
            showarrow=False,
            font=dict(color="#000000", size=10),
            xanchor="left",
            yanchor="bottom"
        ))

        shapes.append(dict(
            type="line",
            xref="x",
            yref="y",
            x0=session_df['time'].iloc[0] if not session_df.empty else session_start_dt,
            y0=line_075,
            x1=session_df['time'].iloc[-1] if not session_df.empty else session_end_dt,
            y1=line_075,
            line=dict(color="#ff9800", width=1, dash="dash"),
            layer="above"
        ))
        annotations.append(dict(
            x=session_9am,
            y=line_075,
            xref="x",
            yref="y",
            text="0.75% RTH Gap",
            showarrow=False,
            font=dict(color="#000000", size=10),
            xanchor="left",
            yanchor="bottom"
        ))

    pd_bsl = pd_ssl = None
    if idx > 0:
        prev_session = unique_sessions[idx - 1]
        prev_row = pd_levels_df[pd_levels_df['session_date'] == prev_session].iloc[0]
        pd_bsl = prev_row['high']
        pd_ssl = prev_row['low']
        pd_bsl_breached = (not session_df.empty) and (session_df['high'] >= pd_bsl).any()
        pd_ssl_breached = (not session_df.empty) and (session_df['low'] <= pd_ssl).any()

        if not pd.isna(pd_bsl):
            shapes.append(dict(
                type="line",
                xref="x",
                yref="y",
                x0=session_df['time'].iloc[0] if not session_df.empty else session_start_dt,
                y0=pd_bsl,
                x1=session_df['time'].iloc[-1] if not session_df.empty else session_end_dt,
                y1=pd_bsl,
                line=dict(color="#801922", width=1, dash="solid"),
                layer="above"
            ))
            session_9am = pd.Timestamp.combine(session_date, pd.to_datetime('12:00:00').time()).tz_localize('America/New_York')
            annotations.append(dict(
                x=session_9am,
                y=pd_bsl,
                xref="x",
                yref="y",
                text="PD BSL",
                showarrow=False,
                font=dict(color="#801922", size=10),
                xanchor="left",
                yanchor="bottom"
            ))
        
        if not pd.isna(pd_ssl):
            shapes.append(dict(
                type="line",
                xref="x",
                yref="y",
                x0=session_df['time'].iloc[0] if not session_df.empty else session_start_dt,
                y0=pd_ssl,
                x1=session_df['time'].iloc[-1] if not session_df.empty else session_end_dt,
                y1=pd_ssl,
                line=dict(color="#801922", width=1, dash="solid"),
                layer="above"
            ))
            session_9am = pd.Timestamp.combine(session_date, pd.to_datetime('12:00:00').time()).tz_localize('America/New_York')
            annotations.append(dict(
                x=session_9am,
                y=pd_ssl,
                xref="x",
                yref="y",
                text="PD SSL",
                showarrow=False,
                font=dict(color="#801922", size=10),
                xanchor="left",
                yanchor="top"
            ))
    
    open_1000 = pd.Timestamp.combine(session_date, pd.to_datetime('10:00:00').time()).tz_localize('America/New_York')
    session_df_after_1000 = session_df[session_df['time'] >= open_1000]
    
    session_9am = pd.Timestamp.combine(session_date, pd.to_datetime('09:29:00').time()).tz_localize('America/New_York')
    session_noon = pd.Timestamp.combine(session_date, pd.to_datetime('12:00:00').time()).tz_localize('America/New_York')
    zoom_df = session_df[(session_df['time'] >= session_9am) & (session_df['time'] <= session_noon)]
    zoom_high = zoom_df['high'].max() if not zoom_df.empty else np.nan
    zoom_low = zoom_df['low'].min() if not zoom_df.empty else np.nan
    y_buffer = (zoom_high - zoom_low) * 0.1 if not np.isnan(zoom_high) and not np.isnan(zoom_low) else 1

    fig = go.Figure()
    if not session_df.empty:
        fig.add_trace(
            go.Candlestick(
                x=session_df['time'],
                open=session_df['open'],
                high=session_df['high'],
                low=session_df['low'],
                close=session_df['close'],
                name=str(session_date),
                increasing_line_color='#000000',
                increasing_fillcolor='#66bb6a',
                increasing_line_width=1,
                decreasing_line_color='#000000',
                decreasing_fillcolor='#000000',
                decreasing_line_width=1
            )
        )

    if rth_gap_range is not None:
        rth_gap_range_str = f"{rth_gap_range:.2f} pts"
    else:
        rth_gap_range_str = "N/A"

    fig.update_layout(
        title=f'{session_date} | RTH Gap Range: {rth_gap_range_str} |',
        xaxis_title='time',
        yaxis_title='Price',
        autosize=True,
        height=600,
        showlegend=False,
        plot_bgcolor='#d0d4de',
        paper_bgcolor='#d0d4de',
        shapes=shapes,
        annotations=annotations,
        yaxis=dict(
            range=[zoom_low - y_buffer, zoom_high + y_buffer] if not np.isnan(zoom_high) and not np.isnan(zoom_low) else None
        ),
        xaxis=dict(
            range=[session_9am, session_noon],
            showgrid=False,
            rangeslider=dict(visible=False)
        ),
    )

    fig.update_xaxes(showgrid=False, rangeslider=dict(visible=False))
    fig.update_yaxes(showgrid=False, tickformat=',.2f', fixedrange=False)

    file_path = os.path.join(save_dir, f'org{session_date}.html')
    fig.write_html(file_path)
    display(IFrame(src=file_path, width='100%', height=700))